In [ ]:
# %load imports.py
%load_ext autoreload
%autoreload 2
%reload_kedro
%config Completer.use_jedi = False  ## (To fix autocomplete)
%matplotlib inline

import pandas as pd
pd.set_option('display.max_columns', 500)
from src.models.vmm import ModelSimulator
import matplotlib.pyplot as plt
from src.visualization.plot import track_plots, plot, captive_plot
import kedro
import numpy as np
import os.path
import anyconfig

import matplotlib
#plt.style.use('paper')
#from IPython.display import set_matplotlib_formats
#set_matplotlib_formats('pdf')

from myst_nb import glue
from src.symbols import *
import src.symbols as symbols
from src.system_equations import *

from IPython.display import display, Math, Latex, Markdown
from sympy.physics.vector.printing import vpprint, vlatex

from src.parameters import df_parameters
p = df_parameters["symbol"]

# Read configs:
conf_path = os.path.join("../../conf/base/")
runs_globals_path = os.path.join(
    conf_path,
    "runs_globals.yml",
)

runs_globals = anyconfig.load(runs_globals_path)
model_test_ids = runs_globals["model_test_ids"]

join_globals_path = os.path.join(
    conf_path,
    "join_globals.yml",
)

joins = runs_globals["joins"]
join_runs_dict = anyconfig.load(join_globals_path)

globals_path = os.path.join(
    conf_path,
    "globals.yml",
)
global_variables = anyconfig.load(globals_path)



vmms = global_variables["vmms"]
from src.visualization.plot import track_plots, track_plot, plot, plot_parameters
#from jb_helpers import df_to_myst, parameter_to_latex
from wPCC_pipeline.presentation import set_file_name, snapshot

In [ ]:
id = 'HSVA_CPMC_KVLCC2_Z_20_10_S'
ship = 'kvlcc2_hsva'
vmm_name = 'vmm_abkowitz'
data = catalog.load(f"{ship}.initial.{id}.data_ek_smooth")
ek = catalog.load(f"{ship}.vmm_linear.ek")
ship_data = catalog.load(f"{ship}.ship_data")

In [ ]:
data['U'] = np.sqrt(data['u']**2 + data['v']**2)
df_sim = ek.simulate(data=data, input_columns=['delta','U'], solver='Radau')

dataframes = {
    'Model test' : data,
    'Initial guessed model': df_sim,
    'Iteration 1': catalog.load(f"{ship}.initial.{vmm_name}.joined.{id}.data_resimulate"),
    'Iteration 2': catalog.load(f"{ship}.updated.{vmm_name}.joined.{id}.data_resimulate"),
}

styles = {
    'Initial guessed model': {'style':'--','lw':1.5},
    'Iteration 1': {'style':'-.','lw':1.5},
    'Iteration 2': {'style':':','lw':2},
    'Model test' : {'style':'-','lw':1.5},
}


keys = ['u','v','r']
error_keys = [f'$\\epsilon({key})$' for key in keys]  
for key, df_ in dataframes.items():
    df_[error_keys]=df_[keys] - data[keys]

fig,ax=plt.subplots()
track_plots(dataframes, lpp=ship_data['L'], beam=ship_data['B'], time_window=[0,70], N=2, styles=styles, ax=ax);
glue("iterations",fig, display=False)
ax.set_title('')


In [ ]:
set_file_name('iterative_kf')

max_length=np.max([len(key) for key in dataframes.keys()])

with plt.xkcd(scale=0.0, randomness=2):
    fig,ax=plt.subplots()
    fig.set_size_inches(4.0,4.0)
        
    for name, df_ in dataframes.items():
        label = name.ljust(max_length) 
        track_plot(df=df_, lpp=ship_data['L'], beam=ship_data['B'], time_window=[0,70], N=2, ax=ax, label=label, 
                   style=styles[name]['style'])
        ax.set_title('')  
        ax.set_xlim(-40,40)
        ax.set_ylim(0,80)
        ax.legend(loc='center right', bbox_to_anchor=(1.90, 0.5))
        
        snapshot()